In [1]:
import torch
torch.__version__

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from torch.autograd import Variable

## Tensor 소개

##### 스칼라(0차원 텐서)

In [2]:
x = torch.tensor(10)
print(x)
print(x.size())
print("{} 차원".format(x.dim()))

tensor(10)
torch.Size([])
0 차원


##### 스칼라(1차원 텐서)

In [3]:
y = torch.rand(10)
print(y)
print(y.size())
print("{} 차원".format(y.dim()))

tensor([0.8256, 0.2412, 0.7703, 0.2570, 0.7848, 0.4233, 0.8342, 0.2768, 0.4032,
        0.6623])
torch.Size([10])
1 차원


##### 벡터(1차원 텐서)

In [4]:
temp = torch.FloatTensor([23, 24, 24.5, 26, 27.2, 23.0])
temp.size()

torch.Size([6])

##### 행렬(2차원 텐서)

In [5]:
from sklearn.datasets import load_boston

boston = load_boston()
print(boston.data.shape)

(506, 13)


In [6]:

boston_tensor = torch.from_numpy(boston.data)
boston_tensor.size()

torch.Size([506, 13])